In [ ]:
R.home()

In [ ]:
baizer::pkglib(tidyverse, baizer, Biostrings)

In [ ]:
sample <- snakemake@wildcards[['sample']]

In [ ]:
airr <- read_tsv(snakemake@input[['igblast_tsv']])

In [ ]:
airr <- airr %>% mutate(germline_alignment_aa_nogap = str_replace_all(germline_alignment_aa, '-', ''))

In [ ]:
write_fasta <- function(x, file) {
  if (is.null(names(x))) {
    names(x) <- seq_along(x)
  }

  res <- map2_chr(names(x), x, ~ str_c(">", .x, "\n", .y))
  out <- file(file, "w")
  writeLines(res, out)
  close(out)
}

In [ ]:
seqs <- airr[['sequence']]

In [ ]:
ds <- DNAStringSet(seqs)

In [ ]:
frame1 <- translate(ds, no.init.codon=TRUE, if.fuzzy.codon='solve')
frame2 <- translate(subseq(ds, start=2), no.init.codon=TRUE, if.fuzzy.codon='solve')
frame3 <- translate(subseq(ds, start=3), no.init.codon=TRUE, if.fuzzy.codon='solve')

In [ ]:
frame1_stop <- str_count(as.character(frame1), '\\*')
frame2_stop <- str_count(as.character(frame2), '\\*')
frame3_stop <- str_count(as.character(frame3), '\\*')

In [ ]:
frame_df <- tibble(frame1_stop=frame1_stop, frame2_stop=frame2_stop, frame3_stop=frame3_stop) %>%
    mutate(min_stop = min(frame1_stop, frame2_stop, frame3_stop)) %>%
    mutate(frame = case_when(
        min_stop==frame1_stop ~ 1,
        min_stop==frame2_stop ~ 2,
        min_stop==frame3_stop ~ 3
    ))

In [ ]:
trans_aa <- translate(subseq(ds, start=frame_df[['frame']]), no.init.codon=TRUE, if.fuzzy.codon='solve')

In [ ]:
trans_aa <- as.character(trans_aa)

In [ ]:
names(trans_aa) <- airr[['sequence_id']]

In [ ]:
write_fasta(trans_aa, snakemake@output[['aa_fa']])

In [ ]:
pull(airr, germline_alignment_aa_nogap, sequence_id) %>% write_fasta(snakemake@output[['gm_aa_fa']])